# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import datetime
import api_token  # handle iexcloud API tokens
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(), api_token.get_api_token()

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [32]:
# stocks = pd.read_csv('sp_500_stocks.csv')

df = pd.read_csv('nasdaq_100_stocks.csv.gz', compression="gzip", names=["Ticker", "Company"], skiprows=[0])
stocks = df["Ticker"].values.tolist()

In [33]:
",".join(stocks)

'ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,ANSS,AAPL,AMAT,ASML,ADSK,ADP,BIDU,BIIB,BMRN,BKNG,AVGO,CDNS,CDW,CERN,CHTR,CHKP,CTAS,CSCO,CTXS,CTSH,CMCSA,CPRT,COST,CSX,DXCM,DOCU,DLTR,EBAY,EA,EXC,EXPE,FB,FAST,FISV,FOXA,FOX,GILD,IDXX,ILMN,INCY,INTC,INTU,ISRG,JD,KDP,KLAC,KHC,LRCX,LBTYA,LBTYK,LULU,MAR,MXIM,MELI,MCHP,MU,MSFT,MRNA,MDLZ,MNST,NTES,NFLX,NVDA,NXPI,ORLY,PCAR,PAYX,PYPL,PEP,PDD,QCOM,REGN,ROST,SGEN,SIRI,SWKS,SPLK,SBUX,SNPS,TMUS,TTWO,TSLA,TXN,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,XEL,XLNX,ZM'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [6]:
symbol = 'AAPL'
api_url = f'{BASE_URL}/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2140879688921,
 'week52high': 140.52,
 'week52low': 57.98,
 'week52change': 0.8536675444223831,
 'sharesOutstanding': 17502018296,
 'float': 0,
 'avg10Volume': 97425432,
 'avg30Volume': 97379952,
 'day200MovingAvg': 112.17,
 'day50MovingAvg': 121.05,
 'employees': 0,
 'ttmEPS': 3.37,
 'ttmDividendRate': 0.8408227339136055,
 'dividendYield': 0.006851113669221946,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-31',
 'nextEarningsDate': '0',
 'peRatio': 37.49537375013228,
 'beta': 1.207948970704424,
 'maxChangePercent': 46.31542825326148,
 'year5ChangePercent': 3.6219492050774127,
 'year2ChangePercent': 2.02324794178756,
 'year1ChangePercent': 0.8413361940171279,
 'ytdChangePercent': 0.679470438501031,
 'month6ChangePercent': 0.4420190691463951,
 'month3ChangePercent': 0.04127947997381534,
 'month1ChangePercent': 0.04826844581281417,
 'day30ChangePercent': 0.04881402594240187,
 'day5ChangePercent': -0.01064269560551317}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [7]:
data['year1ChangePercent']

0.8413361940171279

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,ATVI,84.44,0.549479,N/A
1,ADBE,493.03,0.581309,N/A
2,AMD,94.85,1.339264,N/A
3,ALXN,124.70,0.048830,N/A
4,ALGN,509.69,0.889825,N/A
...,...,...,...,...
98,WBA,42.63,-0.262998,N/A
99,WDAY,222.49,0.347411,N/A
100,XEL,66.46,0.091752,N/A
101,XLNX,147.44,0.597484,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [11]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:21]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,TSLA,632.73,7.985311,N/A
1,MRNA,161.70,7.380596,N/A
2,ZM,402.95,4.952492,N/A
3,PDD,160.66,3.042116,N/A
4,DOCU,226.29,1.993694,N/A
5,MELI,1656.77,1.665953,N/A
6,NVDA,527.03,1.483386,N/A
7,JD,83.52,1.477357,N/A
8,AMD,94.85,1.339264,N/A
9,PYPL,223.88,1.075874,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [12]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [13]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/home/wengong/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,TSLA,632.73,7.985311,75
1,MRNA,161.70,7.380596,294
2,ZM,402.95,4.952492,118
3,PDD,160.66,3.042116,296
4,DOCU,226.29,1.993694,210
5,MELI,1656.77,1.665953,28
6,NVDA,527.03,1.483386,90
7,JD,83.52,1.477357,570
8,AMD,94.85,1.339264,502
9,PYPL,223.88,1.075874,212


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [14]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [15]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0        0.728155
1        0.757282
2         0.92233
3        0.194175
4        0.902913
          ...    
98     0.00970874
99       0.543689
100      0.271845
101      0.747573
102      0.980583
Name: One-Year Return Percentile, Length: 103, dtype: object
0      0.436893
1      0.563107
2       0.92233
3      0.184466
4      0.961165
         ...   
98     0.106796
99     0.582524
100    0.165049
101    0.902913
102    0.970874
Name: Six-Month Return Percentile, Length: 103, dtype: object
0        0.31068
1       0.194175
2       0.485437
3       0.466019
4       0.941748
         ...    
98      0.757282
99      0.330097
100    0.0873786
101     0.912621
102     0.165049
Name: Three-Month Return Percentile, Length: 103, dtype: object
0       0.718447
1       0.378641
2       0.660194
3      0.0776699
4       0.621359
         ...    
98      0.592233
99      0.184466
100    0.0291262
101     0.679612
102    0.0582524
Name: One-Month Return Percentile, Length: 103, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ATVI,86.76,N/A,0.554208,0.728155,0.151900,0.436893,0.038999,0.31068,0.097945,0.718447,N/A
1,ADBE,495.58,N/A,0.590028,0.757282,0.224499,0.563107,0.009261,0.194175,0.027228,0.378641,N/A
2,AMD,92.03,N/A,1.362073,0.92233,0.621812,0.92233,0.098977,0.485437,0.083928,0.660194,N/A
3,ALXN,123.14,N/A,0.047932,0.194175,0.015812,0.184466,0.090631,0.466019,-0.057651,0.0776699,N/A
4,ALGN,520.81,N/A,0.899621,0.902913,0.827831,0.961165,0.578724,0.941748,0.070417,0.621359,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
98,WBA,43.67,N/A,-0.262256,0.00970874,-0.057224,0.106796,0.204613,0.757282,0.062664,0.592233,N/A
99,WDAY,227.17,N/A,0.351899,0.543689,0.228110,0.582524,0.041463,0.330097,-0.017921,0.184466,N/A
100,XEL,68.30,N/A,0.091259,0.271845,0.000620,0.165049,-0.066789,0.0873786,-0.109004,0.0291262,N/A
101,XLNX,144.10,N/A,0.586498,0.747573,0.492827,0.902913,0.416373,0.912621,0.087221,0.679612,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [16]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [17]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [18]:
portfolio_input()

Enter the value of your portfolio:1000000


In [19]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ATVI,86.76,226,0.554208,0.728155,0.151900,0.436893,0.038999,0.31068,0.097945,0.718447,0.548544
1,ADBE,495.58,39,0.590028,0.757282,0.224499,0.563107,0.009261,0.194175,0.027228,0.378641,0.473301
2,AMD,92.03,213,1.362073,0.92233,0.621812,0.92233,0.098977,0.485437,0.083928,0.660194,0.747573
3,ALXN,123.14,159,0.047932,0.194175,0.015812,0.184466,0.090631,0.466019,-0.057651,0.0776699,0.230583
4,ALGN,520.81,37,0.899621,0.902913,0.827831,0.961165,0.578724,0.941748,0.070417,0.621359,0.856796
5,GOOGL,1839.94,10,0.336010,0.504854,0.231160,0.61165,0.151427,0.631068,0.009565,0.300971,0.512136
6,GOOG,1817.16,10,0.339945,0.524272,0.232315,0.621359,0.147942,0.61165,0.012199,0.330097,0.521845
7,AMZN,3251.98,6,0.787918,0.84466,0.202964,0.514563,-0.051513,0.126214,-0.013164,0.213592,0.424757
8,AMGN,238.92,82,0.015170,0.15534,0.037275,0.223301,-0.060224,0.106796,-0.015282,0.194175,0.169903
9,ADI,143.70,136,0.267998,0.417476,0.166417,0.446602,0.243831,0.805825,0.050772,0.495146,0.541262


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [30]:
today = datetime.datetime.now()
xlsx_file = f"002-momentum_strategy_{today.strftime('%Y-%m-%d')}.xlsx"

In [31]:
xlsx_file

'002-momentum_strategy_2020-12-10.xlsx'

In [24]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [27]:
writer.save()